# Spatial Data Manipulation: Raster

## Overview

### Packages 
There are various packages supporting raster data manipulation. In our example, we take advantage of rasterio, numpy, and xarray. The following is the introduction of each Python package.

<a href=https://rasterio.readthedocs.io/en/latest/index.html>`rasterio` </a>reads and writes gridded raster datasets such as satellite imagery and terrain models with formats (e.g., GeoTIFF; *.tiff) and provides a Python API based on Numpy N-dimensional arrays and GeoJSON. 

<a href=https://numpy.org/>`numpy` </a> provides numerous functionalities for scientific purposes. Among them, we utilize this package to calculate grid datasets once they were imported with rasterio, given that grid dataset has rows and columns to represent geographical phenomena.

<a href=http://xarray.pydata.org/en/stable/>`xarray` </a> is a powerful and flexible package for handling multi-dimensional labeled datasets. It extends pandas-like data manipulation to N-dimensional arrays, making it particularly suitable for analyzing and visualizing complex gridded data, such as climate data, satellite imagery, and other forms of raster data. With support for labeled dimensions and coordinates, xarray facilitates operations on data arrays and datasets, providing advanced capabilities for slicing, arithmetic, and aggregation.

This inclusion explains how xarray is leveraged for advanced data manipulation and analysis in the context of raster datasets, complementing the functionalities provided by rasterio and numpy.


### Dataset
In this lecture, we use Landsat 8 imagery, which was downloaded from <a href=https://earthexplorer.usgs.gov/>Earth Explorer</a>. The instructor acquired two major datasets, which were taken on ***June 22, 2021 (representing summer)*** and ***January 16, 2022 (representing winter)***. Given the size of the original dataset (~1GB), two datasets were clipped with the boundary of Champaign County and we only use ***4 bands (i.e., Near-infrared (NIR), Red, Green, and Blue)*** out of 11 bands of Landsat 8, which are below.

<h3><center>Landsat-8 Operational Land Imager & Thermal Infrared Sensor</center></h3>

| Band Number |	Description | Wavelength |	Resolution |
| :-: | :-: | :-: | :-: |
|Band 1	|Coastal / Aerosol |	0.433 to 0.453 µm |	30 meter |
| **Band 2**	| **Visible blue** |	**0.450 to 0.515 µm** |	**30 meter**|
|**Band 3**	|**Visible green** |	**0.525 to 0.600 µm** |	**30 meter**|
|**Band 4**	|**Visible red** |	**0.630 to 0.680 µm** |	**30 meter** |
|**Band 5**	|**Near-infrared** |	**0.845 to 0.885 µm** |	**30 meter** |
|Band 6	|Short wavelength infrared |	1.56 to 1.66 µm |	30 meter |
|Band 7	|Short wavelength infrared |	2.10 to 2.30 µm |	60 meter |
|Band 8	|Panchromatic |	0.50 to 0.68 µm |	15 meter |
|Band 9	|Cirrus |	1.36 to 1.39 µm |	30 meter |
|Band 10 | Long wavelength infrared |	10.3 to 11.3 µm	| 100 meter |
|Band 11 |	Long wavelength infrared |	11.5 to 12.5 µm	 |100 meter |

### Tasks
1. Examine a single band of Landsat 8 imagery
2. Comebine three bands (RGB) to represent TRUE color of the sattelite imagery
3. Comebine three bands (NIR, R, G) to represent FALSE color of the sattelite imagery
4. Calculate Normalized Difference Vegetation Index (NDVI) with NIR band and R band.
5. Classify regions with the NDVI.

In [ ]:
# Import necessary packages
import rasterio as rio
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
# Define relative paths for dataset
summer_path = './data/landsat8_summer' # Stores Landsat 8 imagery was taken June 22, 2021
winter_path = './data/landsat8_winter' # Stores Landsat 8 imagery was taken January 16, 2022

## 1. Import Raster Dataset with `rasterio`

Here, we are importing the Blue band of Landsat 8 summer imagery (LC08_L1TP_023032_20210622_20210629_02_T1_B2.TIF), to see how the raster data is imported and presented in Python. 

One more thing, `os` module is an embedded module in Python, and it provides a portable way of using operating system dependent functionality. For example, we can easily set the file path and call it later with `os.path.join` function. 

In [ ]:
print(f'Summer Data path: {summer_path}')

# You can append the path to the stirng (summer_path) with os.path.join method
file_path = os.path.join(summer_path, f'LC08_L1TP_023032_20210622_20210629_02_T1_B2_Project_Clip.TIF')
print(f'Joined Path: {file_path}')

In [ ]:
# Open a dataset for reading or writing with rasterio(rio).
# Default permission mode is read(r)
b_band_data = rio.open(file_path) 
print(type(b_band_data))
b_band_data

In [ ]:
# You need to use method, read(), to read a dataset’s raw pixels as an numpy N-d array
b_band = b_band_data.read(1)
print(type(b_band)) # Its type is NumPy ndarray (N-dimensional array)
print(b_band.shape) # (M, N): M Rows and N Columns
b_band

Let's display a single band with plt.imshow() method. 

In [ ]:
plt.figure(figsize = (7,7))  # Set the size of figure
plt.imshow(b_band, cmap='Greys_r')  # Display data as an image
plt.colorbar() # Create legend 
plt.show() # Display the map and legend

In [ ]:
# You want to use imshow() method to display an image. The below is to describe an accident if you use plot() method. 
plt.plot(b_band)
plt.show()

In [ ]:
# Check the histogram of the imported image
plt.hist(b_band.flatten(), bins=50)
plt.show()

---
### *Exercise*
1. Import Blue band of Winter Landsat 8 imagery (filename: `LC08_L1TP_023032_20220116_20220123_02_T1_B2.TIF`) which located in `landsat8_winter` folder, and save it as `b_band_winter`.
2. Display the imported image with `plt.imshow()` method. (**Note**: `plt` is `matplotlib` package. We will cover this next week)
---

In [ ]:
# Your code here
## Step 1

## Step 2


## 2. Creating a color image

In the previous example, we rely on a band, so that it cannot be a color image. To display a color image here, we are importing four bands (i.e., Near-infrared (NIR), Red, Green, and Blue), and storing them into ONE NumPy array.

* Blue band: LC08_L1TP_023032_20210622_20210629_02_T1_B2.TIF
* Green band: LC08_L1TP_023032_20210622_20210629_02_T1_B3.TIF
* Red band: LC08_L1TP_023032_20210622_20210629_02_T1_B4.TIF
* NIR band: LC08_L1TP_023032_20210622_20210629_02_T1_B5.TIF

![](https://xarray.dev/xarray-datastructure.png)

In [ ]:
# Import packages
import xarray as xr
import rioxarray as rioxr # rasterio extension for xarray

In [ ]:
# Stack bands into an xarray dataset
def stack_bands(bands, path):
    arrays = []

    # Iterate through the bands
    for band_num in bands.keys():
        band_path = os.path.join(path, f'LC08_L1TP_023032_20210622_20210629_02_T1_B{band_num}.TIF') # Path to the image file
        arr = rioxr.open_rasterio(band_path) # Open the band with rioxarray
        arrays.append(arr) # Append the array to the list
        print(f'Band {band_num} shape: {arr.shape}')

    # Concatenate arrays and assign band labels
    stacked = xr.concat(arrays, dim='band')
    stacked = stacked.assign_coords(band=('band', 
                                          list(bands_dict.values()) # Assign band labels 
                                          )
                                    )  
    
    return stacked

# Define bands and their labels
bands_dict = {2: 'Blue', 3: 'Green', 4: 'Red', 5: 'NIR'}  # Band labels
summer_path = './data/landsat8_summer'  # Example path
arr_summer = stack_bands(bands_dict, summer_path)

arr_summer

In [ ]:
type(arr_summer)

In [ ]:
arr_summer.rio.crs

In [ ]:
arr_summer.shape

In [ ]:
arr_summer.dtype

In [ ]:
# Selecting a specific band
arr_summer.sel(band='Red')

In [ ]:
# Selecting multiple bands
arr_summer.sel(band=['Red', 'Green', 'Blue'])

In [ ]:
# Plot image without setting value limits
arr_summer.sel(band=['Red', 'Green', 'Blue']).plot.imshow(figsize=(7, 7))

In [ ]:
# Plot image with value limits (robust=True)
# This will automatically set the limits based on the data range
arr_summer.sel(band=['Red', 'Green', 'Blue']).plot.imshow(robust=True, figsize=(7, 7))

In [ ]:
# Plot a single band (Blue)
arr_summer.sel(band='Blue').plot.imshow(cmap='Greys_r', figsize=(7, 7))

In [ ]:
# Check the histogram of the imported image
plt.hist(arr_summer.sel(band='Blue').values.flatten(), bins=50)
plt.show()

In [ ]:
# Plot a single band (Blue) with robust scaling
# This will automatically set the limits based on the data range
arr_summer.sel(band='Blue').plot.imshow(robust=True, cmap='Greys_r', figsize=(7, 7))

---
### *Exercise*
1. Investigate the following code for `arr_summer`. Import FOUR BANDS of Winter Landsat 8 imagery (filename: `LC08_L1TP_023032_20220116_20220123_02_T1_B{band_num}.TIF`) which located in `landsat8_winter` folder, and save it as `arr_winter`.

```python
    # Stack bands into an xarray dataset
        def stack_bands(bands, path):
            arrays = []

            # Iterate through the bands
            for band_num in bands.keys():
                band_path = os.path.join(path, f'LC08_L1TP_023032_20210622_20210629_02_T1_B{band_num}.TIF') # Path to the image file
                arr = rioxr.open_rasterio(band_path) # Open the band with rioxarray
                arrays.append(arr) # Append the array to the list
                print(f'Band {band_num} shape: {arr.shape}')

            # Concatenate arrays and assign band labels
            stacked = xr.concat(arrays, dim='band')
            stacked = stacked.assign_coords(band=('band', 
                                                list(bands_dict.values()) # Assign band labels 
                                                )
                                            )  
            
            return stacked

        # Define bands and their labels
        bands_dict = {2: 'Blue', 3: 'Green', 4: 'Red', 5: 'NIR'}  # Band labels
        summer_path = './data/landsat8_summer'  # Example path
        arr_summer = stack_bands(bands_dict, summer_path)

        arr_summer
```
---

In [ ]:
# Your code here (MODIFY THE CODE BELOW)

# Stack bands into an xarray dataset
def stack_bands(bands, path):
    arrays = []

    # Iterate through the bands
    for band_num in bands.keys():
        band_path = os.path.join(path, f'LC08_L1TP_023032_20210622_20210629_02_T1_B{band_num}.TIF') # Path to the image file
        arr = rioxr.open_rasterio(band_path) # Open the band with rioxarray
        arrays.append(arr) # Append the array to the list
        print(f'Band {band_num} shape: {arr.shape}')

    # Concatenate arrays and assign band labels
    stacked = xr.concat(arrays, dim='band')
    stacked = stacked.assign_coords(band=('band', 
                                        list(bands_dict.values()) # Assign band labels 
                                        )
                                    )  
    
    return stacked

# Define bands and their labels
bands_dict = {2: 'Blue', 3: 'Green', 4: 'Red', 5: 'NIR'}  # Band labels
summer_path = './data/landsat8_summer'  # Example path
arr_summer = stack_bands(bands_dict, summer_path)

arr_summer

Given that we have more than three bands (we have four bands in `arr_summer`), we now can assign them to RGB, respectively. In the domain of Remote Sensing, there are two well-known appraoches to display the color image. 

<h3><center> True and False Color representation of Satellite image </center></h3>

| Method | Red Color | Green Color | Blue Color | Purpose |
| :-: | :-: | :-: | :-: | :-: |
| True Color | Red Band | Green Band | Blue Band | True color image |
| False Color | NIR Band | Red Band | Green Band | Monitoring Vegitation |

![True and False Color representation of Satellite image](./data/True_False_Color.png)

Source: Garrard, Chris. (2016). *Geoprocessing with Python*. Manning. p.176

In [ ]:
# Plot multiple bands (NIR, Red, Green) with robust scaling
# This is to visualize the RGB composite with NIR, Red, and Green bands (False color composite)
arr_summer.sel(band=['NIR', 'Red', 'Green']).plot.imshow(robust=True, figsize=(7, 7))

---
### *Exercise*
1. Display TRUE color of the winter imagery with `xarray.plot.imshow()` method. You need to select 'Red', 'Green', and 'Blue' bands from `arr_winter`.
2. Display FALSE color of the winter imagery with `xarray.plot.imshow()` method. You need to select 'NIR', 'Red', and 'Green' bands from `arr_winter`.
---

In [ ]:
# Your code here for task 1


In [ ]:
# Your code here for task 2


## 3. Map Algebra for NDVI

Sometimes, it is challenging to quantify the temporal changes of vegetation with bare eyes. To overcome the issue, we often utilize NDVI (Normalized Difference Vegetation Index). The index quantifies vegetation by measuring the difference between near-infrared (which vegetation strongly reflects) and red light (which vegetation absorbs).

\begin{gather*}
NDVI = \frac{NIR - RED}{NIR + RED}
\end{gather*}

First of all, you need to match the min and max values of the bands, which is called *normalization*. The values of the bands are between 0 and 1. 

In [ ]:
def normalize_xarray(data_array):
    """
    Normalize the values in an xarray DataArray to the range [0, 1].

    Parameters:
    - data_array: xarray.DataArray to be normalized.

    Returns:
    - xarray.DataArray with normalized values.
    """
    min_val = data_array.min()
    max_val = data_array.max()
    
    # Normalize the DataArray
    normalized = (data_array - min_val) / (max_val - min_val)
    
    return normalized


In [ ]:
arr_summer_nir = normalize_xarray(arr_summer.sel(band='NIR'))
arr_summer_nir

In [ ]:
arr_summer_red = normalize_xarray(arr_summer.sel(band='Red'))
arr_summer_red

In [ ]:
# Plot the normalized NIR and Red bands
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

arr_summer_nir.plot.hist(bins=50, ax=ax[0], label='NIR')
arr_summer_red.plot.hist(bins=50, ax=ax[1], label='Red')

fig.tight_layout()
plt.show()

In [ ]:
# Calculate NDVI
# NDVI = (NIR - Red) / (NIR + Red)
arr_summer_ndvi = (arr_summer_nir - arr_summer_red) / (arr_summer_nir + arr_summer_red)
arr_summer_ndvi

In [ ]:
# Remove NaN values
arr_summer_ndvi = arr_summer_ndvi.fillna(0)  # Fill NaN values with 0
arr_summer_ndvi 

In [ ]:
# Visualize NDVI
arr_summer_ndvi.plot.imshow(cmap='Greens', vmin=0, vmax=1, figsize=(7, 7))

---
### *Exercise*
1. Create two xarray (`arr_winter_red` and `arr_winter_nir`) by slicing `arr_winter` with the bands of Red and NIR, respectively. 
2. Normalize the values of `arr_winter_red` and `arr_winter_nir` by using `normalize_xarray` function defined in the cell below.

```python
    def normalize_xarray(data_array):
        min_val = data_array.min()
        max_val = data_array.max()
        
        # Normalize the DataArray
        normalized = (data_array - min_val) / (max_val - min_val)
        
        return normalized
```

3. Calculate NDVI with the normalized bands, save it as `arr_winter_ndvi`, and replace NaN value with 0.
4. Display the NDVI with `xarray.plot.imshow()` method.
---

In [ ]:
# Your code here

# Slice the winter data to get the Red and NIR bands

# Normalize the Red and NIR bands

# Calculate NDVI

# Visualize NDVI



## 4. Classify Raster Image based on the NDVI

Given that NDVI is an index (ranged -1 ~ +1), it helps us to determine the degree of vegetation of each cell. Here, we use NDVI to classify the image and check the area covered by vegetation. 

| NDVI Range | Meaning |
| :-: | :-: |
| ~ 0.1 | Barren rock, sand, or snow |
| 0.2 ~ 0.5 | Sparse vegetation (e.g., shrub, grassland) |
| 0.6 ~ | Dense vegetation (e.g., forest) |

Source: https://www.usgs.gov/special-topics/remote-sensing-phenology/science/ndvi-foundation-remote-sensing-phenology


With <a href=https://numpy.org/doc/stable/reference/generated/numpy.digitize.html>`np.digitize()`</a> function, we can classify the image with a given range (e.g., `bins` below)

In [ ]:
# Define classification bounds
bounds = np.array([-1, 0.15, 0.5, 1])

# Reclassify NDVI using np.digitize within xarray
ndvi_class_summer = xr.apply_ufunc(
    np.digitize,
    arr_summer_ndvi,
    kwargs={'bins': bounds}
)

ndvi_class_summer

In [ ]:
# Check the unique classes
print(f'Classes in the NDVI image: {np.unique(ndvi_class_summer)}')

If we rely on `np.digitize()` function, we need to convert `xarray` to `numpy` array. The `np.digitize()` function is used to classify the NDVI values into discrete bins. 


In [ ]:
arr_summer_ndvi_np = arr_summer_ndvi.to_numpy()
arr_summer_ndvi_np

In [ ]:
np.digitize(arr_summer_ndvi_np, bounds)

If you plot the result of `np.digitize()` directly, it doesn't mean anything. `np.digitize()` returns the index of the range that the value falls into. 

In [ ]:
ndvi_class_summer.plot.imshow(cmap='Greens', figsize=(7, 7))

You can specify colors per class with `matplotlib.colors.ListedColormap`. 

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

ndvi_colors = ["grey", "yellow", "green"]
ndvi_cmap = ListedColormap(ndvi_colors)

# Define class names
ndvi_names = [
    "Low Vegetation",
    "Moderate Vegetation",
    "High Vegetation",
]

fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(ndvi_class_summer, cmap=ndvi_cmap)

# Create custom legend
legend_patches = [Patch(color=ndvi_colors[i], label=ndvi_names[i]) for i in range(len(ndvi_names))]
ax.legend(handles=legend_patches, loc='upper right', title='NDVI Classes')

plt.show()

Let's calculate the area covered by vegetation and its percentage, for both summer and winter.

Reclassify NDVI result for the Winter

In [ ]:
# Define classification bounds
bounds = np.array([-1, 0.15, 0.5, 1])

# Reclassify NDVI using np.digitize within xarray
ndvi_class_winter = xr.apply_ufunc(
    np.digitize,
    arr_winter_ndvi,
    kwargs={'bins': bounds}
)
ndvi_class_winter


Plot NDVI results for both summer and winter

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 7))

ndvi_colors_winter = ["grey", "yellow"]
ndvi_cmap_winter = ListedColormap(ndvi_colors_winter)

axes[0].imshow(ndvi_class_summer, cmap=ndvi_cmap)
axes[1].imshow(ndvi_class_winter, cmap=ndvi_cmap_winter)

# # Create custom legend
legend_patches = [Patch(color=ndvi_colors[i], label=ndvi_names[i]) for i in range(len(ndvi_names))]
axes[0].legend(handles=legend_patches, loc='upper right', title='NDVI Classes')
axes[1].legend(handles=legend_patches, loc='upper right', title='NDVI Classes')

plt.show()

`xr.where()` function is used to create a new xarray DataArray based on a condition. It returns an array of the same shape as the input, where each element is replaced by the corresponding value from the specified array if the condition is met, or by NaN otherwise.

```python
    xr.where(`condition`, 
             `true return value`, 
             `other`)
             

In [ ]:
# Unique classes in the summer NDVI image
np.unique(ndvi_class_summer)

In [ ]:
# Comparision
ndvi_class_summer > 1

In [ ]:
# Create a mask for NDVI values greater than 1
vegi_summer = xr.where(ndvi_class_summer > 1, # Condition
                       1, # Value if True
                       0  # Value if False
                       )

vegi_summer

In [ ]:
vegi_summer.plot.imshow(cmap='Greens', figsize=(7, 7))

Let's calculate the area covered by vegetation and its percentage, for both summer and winter.

In [ ]:
# Number of vegetation pixels in the summer image
# .sum() method only counts the True values, therefore, we can check how many cells satisfy the criteria above.
vegi_summer.sum()

In [ ]:
# By multiplying rows and columns, we can get the total number of grids. 
total_grid = vegi_summer.shape[0] * vegi_summer.shape[1] # Total number of pixels in the image
total_grid

In [ ]:
int(vegi_summer.sum())

In [ ]:
print(f"The number of cells covered by vegetation: {int(vegi_summer.sum())}")
print(f"The percentage covered by vegetation: {round(int(vegi_summer.sum())/total_grid, 2)}%")

You can also find the cell size of the raster image with `xarray.rio.resolution()`. The cell size is 30m x 30m, so that we can calculate the area of each class.

In [ ]:
vegi_summer.rio.resolution()

In [ ]:
cell_size = abs(vegi_summer.rio.resolution()[0])
cell_size

In [ ]:
# You can also take advantage of this appraoch to calculate the area covered by vegetation
(int(vegi_summer.sum() * cell_size**2) / (1000 * 1000)) # Unit is km2

## 5. Export Raster

In [ ]:
# Current data type is int64
ndvi_class_summer.dtype

In [ ]:
# Convert the data type to uint8
ndvi_class_summer = ndvi_class_summer.astype("uint8")
ndvi_class_summer

In [ ]:
# Export to GeoTIFF
ndvi_class_summer.rio.to_raster("./data/ndvi_summer.tif")

# Done